# Generar Matriz de Distancias

### Instrucciones:

1.- Primero cargar en la carpeta de la izquierda los inputs del programa que son los siguientes
> a.- Origenes: Es una capa shapefile de poligonos con los campos que quiero calcular.  
Tiene que tener los nombres de los campos en una columna llamada "idest".  
En general esta capa es una parte del espcul.  
Nuna cargar mas de una zona porque la API gratuita no lo permite.   

> b.- Destinos: Es una capa shapefile de puntos con los destinos.  
Debe tener la columna "Localidad" con las localidades de los puertos, molinos o acopios de la capa.  
  
2.- Una vez cargadas las dos capas, ir al menu "Entorno de ejecución" y seleccionar "Reiniciar y ejecutar todo".  

3.- Una vez hecho esto dejar procesar la info y cuando termine de correr el script, aparecerán tildes en todas las casillas y abajo y aparecerá un archivo excel con el nombre "matrix.xlsx" en la misma carpeta de la izquierda donde cargamos los inputs. Hacer click en los 3 puntitos y descargar el archivo con los resultados.

## Aviso la distancia se encuentra


In [1]:
# !pip install -U google-cloud-aiplatform "shapely<2.0.0"
# !pip uninstall shapely -y
# !pip install "shapely<2.0.0"
# !pip install geopandas==0.10.2 contextily osmnx==1.1.1 matplotlib==3.3
!pip install -U google-cloud-aiplatform
!pip install geopandas contextily osmnx matplotlib shapely fiona

In [2]:
# !pip install geopandas contextily osmnx==1.1.1 matplotlib==3.3
# !pip uninstall fiona -y
# !pip cache purge
# !pip install fiona==1.8.22

In [4]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import datetime as datetime
import os
from pathlib import Path
import time
import requests

In [14]:
try:
  espcul_gdf = gpd.read_file('/content/origenes.shp')

except Exception as e:
  espcul_gdf = gpd.read_file('/content/origenes.gpkg')

print(espcul_gdf.shape)
espcul_gdf.head()

ERROR:fiona._env:/content/origenes.shp: No such file or directory


(1654, 11)


,zona,idest,idlot,cultivo,genotipo,campania,has,fecha_fin,fecha_inicio,has_,geometry
0,BA SDE S,Santa Isabel Cozumel,SIC7,MAIZ,DK7303 TRE,24/25,65.238702,2025-06-30,2024-07-01,65.228002,"MULTIPOLYGON (((-59.27411 -38.58024, -59.27411..."
1,BA OESTE DX,Sauce Iru,SIRLO03,GIRASOL,SYN3970CL,24/25,141.881007,2025-06-30,2024-07-01,141.977541,"MULTIPOLYGON (((-62.20963 -36.54909, -62.22081..."
2,BA OESTE DX,Sauce Iru,SIRME03,TRIGO,B802,24/25,306.900000,2024-12-20,2024-07-01,307.152770,"MULTIPOLYGON (((-62.24571 -36.55146, -62.24566..."
3,BA NORTE,Los Nietos,LN01,TRIGO,Pehuen,24/25,78.700000,2024-12-01,2024-07-01,78.774649,"MULTIPOLYGON (((-61.59836 -34.99056, -61.59844..."
4,BA NORTE,San Federico,SFE03,CEBADA,Andreia,24/25,185.300000,2024-12-02,2024-07-01,185.273936,"MULTIPOLYGON (((-60.83547 -33.82626, -60.83058..."


In [15]:
zonas_list = espcul_gdf.zona.unique()
zonas_list

array(['BA SDE S', 'BA OESTE DX', 'BA NORTE', 'BA OESTE', 'BA COS N',
       'SANTA FE N', 'SALTA', 'BA SDE N', 'LA PAMPA', 'CBA SUR',
       'ENTRE RIOS', 'SAN LUIS', 'LA PAMPA NORTE'], dtype=object)

In [17]:
espcul_gdf = espcul_gdf[(espcul_gdf['zona'].isin(['BA SDE S']))]

In [18]:
espcul_gdf = gpd.GeoDataFrame(espcul_gdf, geometry='geometry').reset_index()
espcul_gdf['geometry'] = espcul_gdf.buffer(0.0001)
espcul_dissolve = espcul_gdf.dissolve(by='idest').reset_index()

print(espcul_dissolve.shape)
espcul_dissolve.head()

<ipython-input-18-dd1a9ebbc1b7>:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_gdf['geometry'] = espcul_gdf.buffer(0.0001)


(30, 12)


,idest,geometry,index,zona,idlot,cultivo,genotipo,campania,has,fecha_fin,fecha_inicio,has_
0,Campanella,"POLYGON ((-59.66223 -38.72098, -59.66222 -38.7...",356,BA SDE S,CAMP01,GIRASOL,SY3970CL,24/25,98.743048,2025-06-30,2024-07-01,98.691351
1,Cerro Destino II,"MULTIPOLYGON (((-59.25592 -37.59832, -59.25592...",97,BA SDE S,Destino_Chico,CEBADA,Sinfonia,24/25,7.800000,2025-01-04,2024-06-01,7.760454
2,Cupalen,"MULTIPOLYGON (((-58.49160 -37.26350, -58.49161...",388,BA SDE S,CUPA01,SOJA 2,DM33e22,24/25,157.455620,2025-06-30,2025-01-01,157.515322
3,Dos Hermanas,"POLYGON ((-59.08046 -37.26251, -59.08047 -37.2...",95,BA SDE S,DH_11,TRIGO,B620,24/25,91.700000,2025-01-07,2024-07-01,91.692051
4,El Alivio,"MULTIPOLYGON (((-59.28175 -38.53114, -59.28176...",66,BA SDE S,EA4,TRIGO,Catalpa,24/25,42.600000,2025-01-06,2024-07-01,42.561650


In [19]:
espcul_dissolve['x']= espcul_dissolve['geometry'].centroid.x
espcul_dissolve['y']= espcul_dissolve['geometry'].centroid.y
espcul_dissolve['centres']= espcul_dissolve['geometry'].centroid
espcul_centroids = espcul_dissolve.loc[:,['idest','x','y','centres']].copy()
espcul_centroids = espcul_centroids.rename(columns = {'centres': 'geometry'})
espcul_centroids = espcul_centroids.to_crs('EPSG:4326')
espcul_centroids.head()

<ipython-input-19-134670d7288a>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['x']= espcul_dissolve['geometry'].centroid.x
<ipython-input-19-134670d7288a>:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['y']= espcul_dissolve['geometry'].centroid.y
<ipython-input-19-134670d7288a>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['centres']= espcul_dissolve['geometry'].centroid


,idest,x,y,geometry
0,Campanella,-59.666189,-38.716063,POINT (-59.66619 -38.71606)
1,Cerro Destino II,-59.255629,-37.591160,POINT (-59.25563 -37.59116)
2,Cupalen,-58.486200,-37.245280,POINT (-58.48620 -37.24528)
3,Dos Hermanas,-59.078169,-37.240109,POINT (-59.07817 -37.24011)
4,El Alivio,-59.263184,-38.521565,POINT (-59.26318 -38.52156)


In [22]:
espcul_centroids.to_file(driver = "GPKG",filename='/content/centoids.gpkg', encoding='utf-8', index=False)
print('Successfully written output file centroids.gpkg')

Successfully written output file centroids.gpkg


In [23]:
espcul_coord = {}

for i in range(0, len(espcul_centroids)):
    origin = espcul_centroids['idest'][i]
    x = espcul_centroids['geometry'][i].x
    y = espcul_centroids['geometry'][i].y
    coords = (y, x)
    espcul_coord[origin]=coords
# espcul_coord

In [38]:
# Opcion para hacer la matriz con todos los destinos
try:
  destinos_gdf = gpd.read_file('/content/destinos.shp')

except Exception as e:
  destinos_gdf = gpd.read_file('/content/destinos.gpkg')


coords_list = []

for i in range(len(destinos_gdf)):
    geom = destinos_gdf['geometry'][i]
    destino = destinos_gdf['Localidad'][i]

    if geom.geom_type == 'Point':
        x, y = geom.x, geom.y
        coords_list.append((destino, y, x))

    elif geom.geom_type == 'MultiPoint':
        # Podés elegir el primer punto o iterar sobre todos
        for pt in geom.geoms:
            x, y = pt.x, pt.y
            coords_list.append((destino, y, x))

    else:
        print(f'Warning: Geometry at index {i} is not Point or MultiPoint: {geom.geom_type}')

# Convert to DataFrame
import pandas as pd
coords_df = pd.DataFrame(coords_list, columns=['Localidad', 'y', 'x'])
print(coords_df.head())







# print(destinos_gdf.shape)
# destinos_gdf.head()

# destinos_coord = {}

# for i in range(0, len(destinos_gdf)):
#     destino = destinos_gdf['Localidad'][i]
#     x = destinos_gdf['geometry'][i].x
#     y = destinos_gdf['geometry'][i].y
#     coords = (y, x)
#     destinos_coord[destino]=coords

# destinos_coord

ERROR:fiona._env:/content/destinos.shp: No such file or directory


      Localidad          y          x
0    9 de Julio -35.443935 -60.884627
1     Arrecifes -34.067515 -60.108692
2          Azul -36.777447 -59.863443
3  Bahia Blanca -38.717605 -62.265447
4        Barker -37.639661 -59.391129


The centroids may be away form actual street network. Use osmnx to find the closest node on OSM network for routing

In [ ]:
west, south, east, north = espcul_gdf.to_crs('EPSG:4326').total_bounds

In [ ]:
G = ox.graph.graph_from_bbox(north, south, east, west)
nodes = G.nodes()

In [ ]:
espcul_coord_snapped = {}
for name, coords in espcul_coord.items():
    node = ox.distance.nearest_nodes(G, coords[1], coords[0])
    info = nodes[node]
    espcul_coord_snapped[name] = (info['y'], info['x'])

espcul_coord_snapped

In [ ]:
import csv
import os
import requests
import time
ORS_API_KEY = '5b3ce3597851110001cf6248211fcc113a28495ebc487cda6358e7f2'

def get_driving_distance(source_coordinates, dest_coordinates):
    parameters = {
    'api_key': ORS_API_KEY,
    'start' : '{},{}'.format(source_coordinates[1], source_coordinates[0]),
    'end' : '{},{}'.format(dest_coordinates[1], dest_coordinates[0])
    }

    response = requests.get(
        'https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)
    if response.status_code == 200:
        data = response.json()
        summary = data['features'][0]['properties']['summary']
        distance = summary['distance']
        return distance/1000
    else:
         print('Request failed.')
         return -9999


In [ ]:
for k1,v1 in espcul_coord_snapped.items():
    for k2,v2 in destinos_coord.items():
        origin = v1
        destino = v2
        distance = get_driving_distance(origin, destino)
        print('Driving distance from {} to {} is {} km'.format(k1, k2, distance))
        time.sleep(2)

## Using Distance Matrix

# Nueva sección

In [ ]:
origins = list(espcul_coord_snapped.values())
destinations = list(destinos_coord.values())
locations = origins + destinations

In [ ]:
sources_index = list(range(0, len(origins)))
destinations_index = list(range(len(origins), len(locations)))

In [ ]:
locations_swapped = [(y,x) for x,y in locations]

In [ ]:

body = {'locations': locations_swapped,
       'destinations': destinations_index,
       'sources': sources_index,
       'metrics': ['distance']}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ORS_API_KEY,
    'Content-Type': 'application/json; charset=utf-8'
}

response = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)


In [ ]:
distances = response.json()['distances']

In [ ]:
distance_matrix = []
for origin_index, (origin_name, origin_coord) in enumerate(espcul_coord_snapped.items()):
    origin_distances = distances[origin_index]
    origin_y, origin_x =  origin_coord
    for dest_index, distance in enumerate(origin_distances):
        dest_name, dest_coord = list(destinos_coord.items())[dest_index]
        dest_y, dest_x =  dest_coord
        distance_km = distance
        distance_matrix.append([origin_name, origin_y, origin_x, dest_name, dest_y, dest_x, distance])

df = pd.DataFrame(distance_matrix,
                  columns =['origin', 'origin_y', 'origin_x', 'destination', 'dest_y', 'dest_x', 'distance'])

In [ ]:
df['km a destino'] = df['distance']/1000
# df.head()

In [ ]:
# df.to_csv('matrix.csv', index=False)
df.to_excel('matrix.xlsx', index=False)